In [49]:
import tensorflow as tf
import numpy as np
import os
import time

In [50]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

 
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [51]:
with open("/Users/pankaj/dev/git/smu/nlp337/marathi/mrityunjay.txt") as f:
    lines = f.readlines()

text = "".join([l for l in lines if l.strip()!=''])

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
# The maximum length sentence we want for a single input in characters
seq_length = 30
examples_per_epoch = len(text)//(seq_length+1)
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

Length of text: 1372948 characters


In [52]:
from nltk import word_tokenize

In [53]:
words = word_tokenize(text)

In [54]:
len(set(words))

35239

In [55]:
from collections import Counter
c = Counter(words)

In [56]:
import operator
x = c
sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse=True)

In [57]:
sorted_x

[('.', 11379),
 ('!', 9384),
 (',', 8636),
 ('‘', 5324),
 ('’', 3439),
 ('मी', 3209),
 ('?', 2645),
 ('आिण', 2176),
 ('आहे', 2129),
 ('ते', 1989),
 ('तो', 1915),
 ('मला', 1893),
 ('होता', 1680),
 ('\U00102861या', 1605),
 ('नाही', 1583),
 ('होतं', 1479),
 ('काय', 1335),
 ('मा\U0010285eया', 1322),
 ('पण', 1265),
 ('एक', 1256),
 ('होती', 1161),
 ('या', 1139),
 ('होते', 1093),
 ('”', 1028),
 ('ी', 1022),
 ('हे', 1006),
 ('तर', 941),
 ('ती', 922),
 ('–', 914),
 ('की', 896),
 ('आप\U0010286bया', 863),
 ('ा', 845),
 (':', 841),
 ('\U00102861या\U0010285cया', 828),
 ('\U00102861याला', 769),
 ('\U00102861यानं', 734),
 ('आता', 729),
 ('\U00102869हणून', 644),
 ('सव\U00102712', 635),
 ('पर्', 569),
 ('माझं', 519),
 ('असं', 517),
 ('तरी', 501),
 ('एका', 500),
 ('ं', 484),
 ('न\U0010286dहतं', 484),
 ('क\U001028f6न', 481),
 ('तू', 480),
 ('आज', 464),
 ('हात', 459),
 ('घेऊन', 457),
 ('*', 440),
 ('का', 436),
 ('कण\U00102712', 428),
 ('काही', 428),
 ('लागले', 425),
 ('ू', 423),
 ('हा', 414),
 ('ही', 404)

In [58]:
len(words)

268084

In [59]:
len(vocab)

184

In [60]:
x1 = Counter(text)
sorted_x1 = sorted(x1.items(), key=operator.itemgetter(1), reverse=True)
sorted_x1

[(' ', 201446),
 ('ा', 164571),
 ('त', 60872),
 ('ी', 48952),
 ('र', 48444),
 ('य', 46521),
 ('ं', 45440),
 ('न', 40441),
 ('ल', 39702),
 ('े', 38806),
 ('क', 37535),
 ('व', 34470),
 ('ह', 33721),
 ('स', 28661),
 ('म', 28350),
 ('प', 25572),
 ('च', 24455),
 ('ि', 23890),
 ('ो', 22533),
 ('\n', 22402),
 ('ण', 20496),
 ('ु', 17370),
 ('द', 14891),
 ('ग', 14421),
 ('आ', 14303),
 ('ू', 13589),
 ('ज', 13127),
 ('.', 12853),
 ('ड', 11149),
 ('्', 10969),
 ('अ', 10633),
 ('\U00102861', 10440),
 ('श', 10219),
 ('!', 9384),
 ('\U00102712', 8868),
 (',', 8637),
 ('\U0010285c', 8124),
 ('ळ', 8060),
 ('ट', 7814),
 ('ध', 7269),
 ('भ', 6634),
 ('ब', 6411),
 ('ख', 5381),
 ('ठ', 5329),
 ('‘', 5324),
 ('\U00102870', 4944),
 ('घ', 4665),
 ('थ', 4230),
 ('\U0010286b', 4132),
 ('झ', 3839),
 ('उ', 3820),
 ('ए', 3779),
 ('\U00102869', 3496),
 ('’', 3439),
 ('\U001028f6', 2886),
 ('\U00102855', 2842),
 ('?', 2645),
 ('\U0010286d', 2547),
 ('ृ', 2540),
 ('\U0010286f', 2383),
 ('ढ', 2216),
 ('\U0010285e', 2153

In [61]:
Counter(text)

Counter({'म': 28350,
         'ृ': 2540,
         '\U00102861': 10440,
         'य': 46521,
         'ु': 17370,
         'ं': 45440,
         'ज': 13127,
         '\n': 22402,
         '\U001011f0': 4,
         'ि': 23890,
         'श': 10219,
         'व': 34470,
         'ा': 164571,
         'ी': 48952,
         ' ': 201446,
         'स': 28661,
         'त': 60872,
         '©': 1,
         'प': 25572,
         'र': 48444,
         '्': 10969,
         'क': 37535,
         'न': 40441,
         'ल': 39702,
         'अ': 10633,
         'े': 38806,
         'ह': 33721,
         ',': 8637,
         '\U00102867': 301,
         'ग': 14421,
         'ऊ': 1570,
         '१': 41,
         '९': 55,
         '४': 10,
         'द': 14891,
         'ठ': 5329,
         'ड': 11149,
         'ॉ': 7,
         'ण': 20496,
         '३': 8,
         '०': 27,
         '.': 12853,
         '‘': 5324,
         '२': 20,
         '-': 1159,
         '७': 8,
         '६': 14,
         'E': 1,
         'm'

In [62]:
# Batch size
BATCH_SIZE = 64

BATCH_SIZE = 16


# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# The embedding dimension
#embedding_dim = 16

# Number of RNN units
#rnn_units = 32


In [63]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
model.summary()
model.compile(optimizer='adam', loss=loss)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (16, None, 256)           47104     
_________________________________________________________________
gru_5 (GRU)                  (16, None, 1024)          3938304   
_________________________________________________________________
dense_5 (Dense)              (16, None, 184)           188600    
Total params: 4,174,008
Trainable params: 4,174,008
Non-trainable params: 0
_________________________________________________________________


In [64]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 30, 184) # (batch_size, sequence_length, vocab_size)


In [65]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
vexample_batch_loss  = loss(target_example_batch, example_batch_predictions)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_mar'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [66]:
EPOCHS=10
#EPOCHS=5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Train for 2768 steps
Epoch 1/10
2768/2768 [==============================] - 1205s 435ms/step - loss: 2.0953
Epoch 2/10
2768/2768 [==============================] - 1082s 391ms/step - loss: 1.7233
Epoch 3/10
2768/2768 [==============================] - 1008s 364ms/step - loss: 1.6323
Epoch 4/10
2768/2768 [==============================] - 9572s 3s/step - loss: 1.5810
Epoch 5/10
2768/2768 [==============================] - 929s 336ms/step - loss: 1.5473
Epoch 6/10
2768/2768 [==============================] - 920s 332ms/step - loss: 1.5256
Epoch 7/10
2768/2768 [==============================] - 921s 333ms/step - loss: 1.5119
Epoch 8/10
2768/2768 [==============================] - 924s 334ms/step - loss: 1.5063
Epoch 9/10
2768/2768 [==============================] - 928s 335ms/step - loss: 1.5060
Epoch 10/10
2768/2768 [==============================] - 895s 323ms/step - loss: 1.5094


In [67]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints_mar/ckpt_10'

In [68]:
model2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model2.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model2.build(tf.TensorShape([1, None]))
model2.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            47104     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_6 (Dense)              (1, None, 184)            188600    
Total params: 4,174,008
Trainable params: 4,174,008
Non-trainable params: 0
_________________________________________________________________


In [69]:
#print(generate_text(model, start_string=u"ROMEO: "))

print(generate_text(model2, start_string=u"जाणार"))


जाणार! ते
छायेचाही अव􂡰थेत दानवमा􂜒ल, अिवशे पटलून िमळाले न􂡭हते. शु􂢝ीवर
बसले होते. 􂡡यांतील पी 􂡡या􂡜या दीन एका िन:श􂡧द भुवया
उडिवला! तो सुखदा आत ज􂡤मला!
‘‘या सताना मला जीवनातील गो􂡫प 􂡩हणजे ब्र􂢻से! गु􂣶 दर् ोणाचं ी अनुथ􂣶 ू कुंतीदेवी – दानशू􂡤य तूरासह अं􂡡कार मराठी! आडवानं􂜒 ा चंदार
िदसले􂡫या... धातर् ी एकिच􂡯णाचं 􂜒
कणाप􂜒 ाठी आणलेलं दु:ख िलघून मला 􂡩हून
गेला. 􂡡याचे तो मराठी तयारा होता! मुख
बरोबर
घेणार होता! तो अमृताश झालेला कुणाचा सैिनक मना􂡜या शुभ्र फुगवून तो वाड􂜐ावर प्रव शोधून के ले. काना􂡜या फु रस
िनराझे उडीिकत आपला
िदि􂡚वजयी मान उंच? छे, कुठंही पांडवानं ा पडला आहे अंगराज कण􂜒ऽऽ आवाजा􂡜या भा􂡭यासाठी – का􂣶न द􂡚ध वष􂜒वू राजदंड आपोआप उंच उडाले. तलवती गंगे परत आलाच तसं जवून पुन:पु􂡤हा 􂡡याचीच
नाही कृ तवत:च, सािमश􂡰तत्राचे पट्टे आलं!
‘‘आ􂡖ाधार􂜒 ानं ी मला अनुलवलं जातं! जीव􂡮यक होते. सवांत􂜒 वायला आला आहे.! 􂡡याचा पांडव
खांदेवढी िफरवत
गदा! रथा􂢘ी सलो􂡫होयु􂢝 तेच कळे.
‘‘तु􂡞याशी थांबिवला होता मी 􂡡या􂡜याशी लढलो. 􂡡याची-
पवाऐका एकमेकांवर घामाचे बथक रथिव􂣶न फळं चतु􂣶्रदेन घेऊन परत मोकळी वळणं घायाळ
होऊ लागला. भीम-गदे􂡜या स􂡭हांनाहीस􂡤मळ

In [70]:
print(generate_text(model2, start_string=u"काही"))

काही पडदल तोडणारी एका शु􂢝ाचा डतू धनु􂡯य
फेकून िदली! संतोष 􂢢ाव मलालाच असं 􂡡यां􂡜या
िश􂡫पातुमांनी तो गोल अ􂡛यां􂡜च सूय􂜒पुत्राची िचव􂣵िवतभर कु􂣵􂡕ेत्र आिण दया शत करीत बपु􂡤हा पण􂜒कु टीसमोर
के वळ आप􂡫याकडे हळूहळू पात्रं घेऊन कसंतरीच
थरथ􂣶 लागला! ‘‘मातािम􂜒पून तळपू फुगला! हरीत आणून राहणार न􂡭हता! 􂡡याला मग तो
नेहमी आपलं जीवनाला जगून ियशेवीट􂜐ा सवां􂜒वर एक
स􂡥तमानही आहे, हे माझे हात होते!
िदवस􂜉िदवस असेल तं ते 􂢢ूतानं?
पिहला प्रय􂡡न 􂡡यां􂡜या अंगाल􂜒 की तु􂡞या दाट
भावनांप􂢘ी भीमपुतर् ां􂡜या संकेत ठाकलेला िदि􂡚वजयमानं मावळ􂡡या 􂡰त􂡧धा􂡜या
समवाराचा.’’
‘‘दुया􂜇धनासाठी काय-􂡰फुंदताच होणार नाही. तो कुणाचाच बोल सोनेरी बुंज-
पायदंडया बंधू􂡜या मुखावर चढून,
अपाप, यमुना सहज तडका मा􂡞या डो􂡬यां􂡜या
पा􂡬याकडे धावू लागलो.
पांडवां􂡜या 􂢣ारात कास मी
􂡕ितृहयेत काय ते समज करायचं न􂡭हतं का?
􂡝या वेळी 􂡡याचे चमकारे
वेधनापुढं
सरळ िन􂡬याभ्रमाला घेऊन मी तजागला!
वाकावत हे अ􂡕रश: जाणार न􂡭ह􂡡या.
‘अजु􂜒नांनी पाठवून ठेवून! मला एक-उलदा􂡜या मरासमुद􂜒 यातना यांचा डताचे सव􂜒 सामा􂡤य आहे! तु􂡩हाला आ􂡖ाधारक जण आता
केवळ जरदैय खड्गं दरू देतो मीही
􂡡या􂡜या आवाजा􂡜या मना􂡜या हळे फुटला!